In [1]:
import pandas as pd
import numpy as np
import re
import json
import string
import spacy

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English


In [2]:
punctuations = string.punctuation
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS
parser = English()

In [ ]:

def spacy_tokenizer(sent):
    tokens = parser(sent)
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations]
    return tokens


def clean_text(text):
    return text.strip().lower()


class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

In [ ]:
spacy_tokenizer('Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.')

In [ ]:
with open("_data/reviews.json", encoding='utf-8', errors='ignore') as json_data:
     data = json.load(json_data, strict=False)

In [ ]:
#data

In [ ]:
reviews=pd.DataFrame.from_dict([v for v in data['reviews']], orient='columns', dtype=None, columns=None)

In [ ]:
#reviews.head()

reviews.info()

#reviews.stars.value_counts()


In [ ]:
reviews = reviews[reviews.stars != 3]

In [ ]:
reviews['sentiment'] = np.where(reviews.stars>3, 1, 0)

In [ ]:
reviews.sentiment.value_counts()

#positivos sobre representados
#A) resampleo

In [ ]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [ ]:
from sklearn.model_selection import train_test_split

X = reviews['text']
ylabels = reviews['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)


In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', tfidf_vector),
                 ('classifier', classifier)])


pipe.fit(X_train,y_train)

In [ ]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

In [3]:
import pickle

In [4]:
pickle.dump(pipe, open("c1_m1.pkl", 'wb'))

NameError: name 'pipe' is not defined

AttributeError: Can't get attribute 'predictors' on <module '__main__'>

In [ ]:
result = pipe.predict("I dont really like it, they were too bad people")

In [ ]:
result